# Fighting Gambling addiction by gambling Smarter
People start playing the lottery for fun, but some people eventually find this activity addictive and become compulsive gamblers who gambles away their savings and loans, and desperately participate in illegal behaviours such theft. 
 A medical institue aims to prevent and treat gambling addiction by building a mobile app that helps lottery addicts better estimate their chances of winning. A Software engineering team will build the app but they need assistance in creating the logical core and calculating the probabilities. 
For the fist version of the app, we will be focusing on the 6/49 lottery and build functions that would allow users to answer the following questions: 
- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different - tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

First, we'll create two functions: a function that calculates factorials, and one calculates combinations.

In [2]:
#Factorial function
def factorial(n):
    final_product = 1
    for i in range(n, 0, -1):
        final_product *= i
    return final_product

#Combination function
def combinations(n, k):
    numerator = factorial(n)
    denominator = factorial(k) * factorial(n-k)
    
    return numerator/denominator


In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. If only one number differs, he doesn't win.

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
- The engineering team wants the function to print the probability value in a friendly way— in a way that people without any probability training are able to understand.


In [3]:
#Creating function that calculated the probability of a user's 6 number combination.
def one_ticket_probability(usr_6_nums):
    num_possible_combos = combinations(49, 6)
    one_ticket_prob = 1/num_possible_combos
    percent_form = one_ticket_prob * 100
    msg = "The probability of your chosen combination {} is {: 7f}%. That means you have a 1 in {:,} chances of winning."
    print(msg.format(usr_6_nums, percent_form, int(num_possible_combos)))

In [4]:
#Testing the one_ticket_probability function.
test1_combos = [1, 2, 3, 4, 5, 6]
one_ticket_probability(test1_combos)

The probability of your chosen combination [1, 2, 3, 4, 5, 6] is  0.000007%. That means you have a 1 in 13,983,816 chances of winning.


In [5]:
#Print the number of rows and columns in the dataset.

Time to open the 649 csv file and save it as a dataset in Pandas.

In [6]:
import pandas as pd

#Opening the 649.csv file and saving it as a dataframe.
lottery6_49 = pd.read_csv('649.csv')

#Print the number of rows and columns in the dataset.
print('Number of rows:', lottery6_49.shape[0])
print('Number of columns:', lottery6_49.shape[1])

Number of rows: 3665
Number of columns: 11


In [7]:
#Exploring the first 3 rows of the dataset.
lottery6_49.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


In [8]:
#Exploring the last 3 rows of the dataset.
lottery6_49.tail(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


In [9]:
def extract_numbers(row):
    row = row[4:10]
    winning_nums = set(row)
    return winning_nums

winning_numbers = lottery6_49.apply(extract_numbers, axis=1)
winning_numbers.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

In [10]:
def check_historical_occurence(user_nums, winner_nums):
    user_nums = set(user_nums)
    winners = user_nums == winner_nums
    wins = winners.sum()
    if wins == 0:
        print('Your number sequence{} never occurred. That does not mean it will never occur. The chances of {} occurring is still 0.0000072%'.format(user_nums, user_nums))
    else:
        print ('Your number sequence{} occurred {} times. That does not mean it will never occur. The chances of {} occurring is still 0.0000072%'.format(user_nums, wins, user_nums))
    
user_set1 = [2, 5, 7, 13, 27, 31]

check_historical_occurence(user_set1, winning_numbers)
    

Your number sequence{2, 5, 7, 13, 27, 31} never occurred. That does not mean it will never occur. The chances of {2, 5, 7, 13, 27, 31} occurring is still 0.0000072%


# Computing the Multi-ticket probability.

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning — on this screen, we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

I've spoken with the engineering team and they gave us the following information:

- The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
- Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
- The function should print information about the probability of winning the big prize depending on the number of different tickets played.


In [11]:
def multi_ticket_probability(n_tickets):
    n_combos = combinations(49, 6)
    prob_success = n_tickets / n_combos
    success_pct_form = prob_success*100
    prob_simplified = int(round((n_combos/n_tickets), 0))
    
    if n_tickets == 1:
        msg = '''The probability of you winning the big prize by purchasing {:,} ticket is {:.7f}%. Which means you have a 1 in {} chance of winning the big prize.'''
        
    else:
        msg = '''The probability of you winning the big prize by purchasing {:,} tickets is {:.6f}%. Which means that you have a 1 in {} chance of winning the big prize.'''
   
    print(msg.format(n_tickets, success_pct_form, prob_simplified))
    
test_inputs = [1, 10, 100, 10000, 1000000, 6991908, 13983816]

for test in test_inputs:
    multi_ticket_probability(test)

The probability of you winning the big prize by purchasing 1 ticket is 0.0000072%. Which means you have a 1 in 13983816 chance of winning the big prize.
The probability of you winning the big prize by purchasing 10 tickets is 0.000072%. Which means that you have a 1 in 1398382 chance of winning the big prize.
The probability of you winning the big prize by purchasing 100 tickets is 0.000715%. Which means that you have a 1 in 139838 chance of winning the big prize.
The probability of you winning the big prize by purchasing 10,000 tickets is 0.071511%. Which means that you have a 1 in 1398 chance of winning the big prize.
The probability of you winning the big prize by purchasing 1,000,000 tickets is 7.151124%. Which means that you have a 1 in 14 chance of winning the big prize.
The probability of you winning the big prize by purchasing 6,991,908 tickets is 50.000000%. Which means that you have a 1 in 2 chance of winning the big prize.
The probability of you winning the big prize by purc

# The Probability of finding 2, 3, 4, and 5 matching numbers. 

In most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:
- Inside the app, the user inputs:
    - six different numbers from 1 to 49; and
    - an integer between 2 and 5 that represents the number of winning numbers expected. 
  
First we need to differentiate between these 2 probability questions:
1. What is the probability of having exactly five winning numbers?
2. What is the probability of having at least five winning numbers?



In [12]:
def probability_less_6(less_6_combo):
    n_possibles_out_of_6 = combinations(6, less_6_combo) #prob out of 6 numbers
    p_less_6 = combinations(43, 6-less_6_combo)
    successful_outcomes = n_possibles_out_of_6 * p_less_6
    
    total_n_combos = combinations(49, 6)
    probability = successful_outcomes/total_n_combos
    
    probability_perc = probability * 100
    combos_simplified = round(total_n_combos/ successful_outcomes)
    
    msg = 'The probability of your combination matching {} out of a possible 6 number combinations is {:.7f}%. This means that you have a 1 in {:,} chances of winning.'
    print(msg.format(less_6_combo, probability_perc, combos_simplified)) 

for i in range(2, 6):
    probability_less_6(i)
    print('-------------------------------------------------------------------------')

    

The probability of your combination matching 2 out of a possible 6 number combinations is 13.2378029%. This means that you have a 1 in 8 chances of winning.
-------------------------------------------------------------------------
The probability of your combination matching 3 out of a possible 6 number combinations is 1.7650404%. This means that you have a 1 in 57 chances of winning.
-------------------------------------------------------------------------
The probability of your combination matching 4 out of a possible 6 number combinations is 0.0968620%. This means that you have a 1 in 1,032 chances of winning.
-------------------------------------------------------------------------
The probability of your combination matching 5 out of a possible 6 number combinations is 0.0018450%. This means that you have a 1 in 54,201 chances of winning.
-------------------------------------------------------------------------


# App Next Steps
For the first version of our app, we've written four main functions for our app:
- one_ticket_probability() — calculates the probability of winning the big prize with a single ticket.
- check_historical_occurrence() — checks whether a certain combination has occurred in the Canada lottery data set.
- multi_ticket_probability() — calculates the probability for any number of of tickets between 1 and 13,983,816.
- probability_less_6() — calculates the probability of having two, three, four or five winning numbers

Possible features for a second versin of the app include:
- Making the outputs even easier to understand by adding fun analogies (for example, we can find probabilities for strange events and compare with the chances of winning in lottery; for instance, we can output something along the lines "You are 100 times more likely to be the victim of a shark attack than winning the lottery").
- Combining the one_ticket_probability() and check_historical_occurrence() to output information on probability and historical occurrence at the same time.
- Creating a function similar to probability_less_6() which calculates the probability of having at least two, three, four or five winning numbers. Hint: the number of successful outcomes for having at least four winning numbers is the sum of these three numbers:
    - The number of successful outcomes for having four winning numbers exactly
    - The number of successful outcomes for having five

In [18]:
strange_possibilities = {'struck by lightning': 700000, 'dying in a plane crash': 60000000
                         , 'dying in a tornado':5000000, 'getting hit by a meteorite': 3000
                         , 'getting killed by a shark':3700000}

def probability_less_6(less_6_combo):
    n_possibles_out_of_6 = combinations(6, less_6_combo) #prob out of 6 numbers
    p_less_6 = combinations(43, 6-less_6_combo)
    successful_outcomes = n_possibles_out_of_6 * p_less_6
    
    total_n_combos = combinations(49, 6)
    probability = successful_outcomes/total_n_combos
    
    probability_perc = probability * 100
    combos_simplified = round(total_n_combos/ successful_outcomes)
    
    if combos_simplified 
        msg = 'The probability of your combination matching {} out of a possible 6 number combinations is {:.7f}%. This means that you have a 1 in {:,} chances of winning.'
        print(msg.format(less_6_combo, probability_perc, combos_simplified)) 


for i in range(2, 6):
    probability_less_6(i)
    print('-------------------------------------------------------------------------')

    

The probability of your combination matching 2 out of a possible 6 number combinations is 13.2378029%. This means that you have a 1 in 8 chances of winning.
-------------------------------------------------------------------------
The probability of your combination matching 3 out of a possible 6 number combinations is 1.7650404%. This means that you have a 1 in 57 chances of winning.
-------------------------------------------------------------------------
The probability of your combination matching 4 out of a possible 6 number combinations is 0.0968620%. This means that you have a 1 in 1,032 chances of winning.
-------------------------------------------------------------------------
The probability of your combination matching 5 out of a possible 6 number combinations is 0.0018450%. This means that you have a 1 in 54,201 chances of winning.
-------------------------------------------------------------------------
